In [10]:
#import usual packages
import numpy as np
import pandas as pd
import requests as rq
import os
import re

In [ ]:
#call the page and display the response => bullshit
page = rq.get('https://finance.yahoo.com/screener/predefined/all_cryptocurrencies_us/')
page_content = page.content
page_content

In [ ]:
#use BeautifulSoup to try better => bullshit
from bs4 import BeautifulSoup
page_BS = BeautifulSoup(page_content, 'html.parser')
page_BS

In [ ]:
a_crypto_names = page_BS.find_all("a", class_=["Fw(600)", "C($linkColor)"])
a_crypto_names_text = [a_crypto_name.get_text() for a_crypto_name in a_crypto_names]
a_crypto_names_text[0]

Scraping Yahoo finances crypto pages to retrieve list of crypto currencies symbols

SELENIUM


In [ ]:
# from selenium import webdriver

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  # Ensure webdriver-manager is installed
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

# Set Chrome options if necessary
options = webdriver.ChromeOptions()

# Automatically download and use the correct ChromeDriver version
service = Service(ChromeDriverManager().install())  # This sets the correct ChromeDriver path


driver = webdriver.Chrome(service=service, options=options)

driver.get('https://finance.yahoo.com/screener/predefined/all_cryptocurrencies_us/')

#submodules and classes within selenium.webdriver.common
from selenium.webdriver.common.by import By

#set up Webdriverwait and support classes
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 5)



In [34]:
#access to scrollDown_button and click to agree on main page 

scrollDown_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#scroll-down-btn')))
scrollDown_button.click()

okDataPrivacy = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#consent-page > div > div > div > form > div.wizard-body > div.actions.couple > button.btn.secondary.reject-all')))
okDataPrivacy.click()

In [ ]:
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Assuming driver and wait are already defined
actions = ActionChains(driver)

crypto_final_list= []

def crypto_list(crypto_names):
    """Generator to fetch crypto names from paginated pages."""
    while True:
        try:
            # Yield current crypto names
            yield crypto_names
            
            
            # Find and interact with the Next button
            next_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 
                    '#scr-res-table > div.W\(100\%\).Mt\(15px\).Ta\(end\) > button.Va\(m\).H\(20px\).Bd\(0\).M\(0\).P\(0\).Fz\(s\).Pstart\(10px\).O\(n\)\:f.Fw\(500\).C\(\$linkColor\) > span > span'
                ))
            )
            
            actions.move_to_element(next_button).perform()
            next_button.click()

            driver.refresh()
            
            # Wait for the next page to load
            crypto_names = [elt.text for elt in wait.until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '[data-test="quoteLink"]'))
            )]


        except Exception as e:
            print('Error:', e)
            break  # Exit the loop on error

# Initialize scraping
cryptos_a_list = wait.until(EC.visibility_of_all_elements_located(
    (By.CSS_SELECTOR, '[data-test="quoteLink"]')
))
initial_crypto_names = [elt.text for elt in cryptos_a_list]

# Use generator
crypto_generator = crypto_list(initial_crypto_names)


# Consume the generator
for crypto_names in crypto_generator:
    crypto_final_list.extend(crypto_names)  # Add scraped names to final list
    print("Current page cryptos:", crypto_names)

# Print the complete list
print("All cryptocurrencies:", crypto_final_list)



In [39]:
import pandas as pd

crypto_final_list = pd.DataFrame(crypto_final_list) 
crypto_final_list.to_csv("list_cryptos_yf.csv")